# Intuitive Evaluation of Transformer-Based Models

In [1]:
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

# NepaliBERT
Model is pretrained in the folder `NepaliBERT`. 
https://huggingface.co/Rajan/NepaliBERT

## Model Description

### Loading Model and Tokenizer

In [ ]:
from transformers import BertTokenizer
vocab_file_dir = './NepaliBERT/' 
bert_tokenizer = BertTokenizer.from_pretrained(vocab_file_dir,
                                     strip_accents=False,
                                      clean_text=False )

In [ ]:
from transformers import BertForMaskedLM
bert_model = BertForMaskedLM.from_pretrained('./NepaliBERT')

In [ ]:
from transformers import pipeline
bert_unmasker = pipeline('fill-mask', model=bert_model, tokenizer=bert_tokenizer)

In [ ]:
bert_unmasker('कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले [MASK] बनाएको छ')

In [ ]:
unmasker('एक दिन बिहानै साढे चार बजे हल्का पेट [MASK]।')

In [ ]:
unmasker('विध्वंसकारी क्षतिको चिन्ताले [MASK] बनाएको छ।')

In [ ]:
unmasker('विध्वंसकारी क्षतिको चिन्ताले [MASK]')

In [ ]:
unmasker('कथाको प्रस्तुति, [MASK] र पात्रहरुको अभिनयले फिल्ममा देखाइएको संसारलाई दर्शकले सजिलै विश्वास गरिरहेका हुन्छन् ।')

# Nepali DistilBERT

Model at HuggingFace 'Sakonii/distilbert-base-nepali'. https://huggingface.co/Sakonii/distilbert-base-nepali. This model is pretrained in folder `distilbert-base-nepali`

## Model Description

### Loading Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer
dbert_tokenizer = AutoTokenizer.from_pretrained('./distilbert-base-nepali/')

In [ ]:
from transformers import AutoModel, DistilBertForMaskedLM
dbert_model = DistilBertForMaskedLM.from_pretrained('./distilbert-base-nepali')

In [ ]:
dbert_unmasker = pipeline('fill-mask', model=dbert_model, tokenizer=dbert_tokenizer)

In [ ]:
dbert_unmasker('कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ')

# Nepali RoBERTa

Model at HuggingFace 'amitness/nepbert'. https://huggingface.co/amitness/nepbert

## Model Description

In [ ]:
roberta_unmasker = pipeline('fill-mask', model='amitness/nepbert', tokenizer='amitness/nepbert')

In [ ]:
roberta_unmasker('कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ')

# TransformerModel Class - A Wrapper to Implement Transformer-based Pretrained Models under study

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, BertForMaskedLM

import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 1000

# Identifiers for models
BERT_MODEL_ID = "BERT"
ROBERTA_MODEL_ID = "RoBERTa"
DISTILBERT_MODEL_ID = "DistilBERT"
DEBERTA_MODEL_ID = "DeBERTa"
MBERT_MODEL_ID = "mBERT"
XLMROBERTA_MODEL_ID = "XLMRoBERTa"
HINDIROBERTA_MODEL_ID = "HindiRoBERTa"

# Identifiers for HuggingFace
HF_BERT = 'Rajan/NepaliBERT'
HF_ROBERTA = 'amitness/nepbert'
HF_DISTILBERT = 'Sakonii/distilbert-base-nepali'
HF_DEBERTA = 'Sakonii/deberta-base-nepali'
HF_XLM_ROBERTA = 'xlm-roberta-base'
HF_M_BERT = 'bert-base-multilingual-uncased'
HF_HIN_ROBERTA = 'flax-community/roberta-hindi'

class TransformerModel:
    
    def __init__(self, model_id):
        
        hf_id = model_id
        self.model_id = model_id
        
        if model_id == BERT_MODEL_ID:
            hf_id = HF_BERT
            
        if model_id == ROBERTA_MODEL_ID:
            hf_id = HF_ROBERTA
                
        if model_id == DISTILBERT_MODEL_ID:
            hf_id = HF_DISTILBERT
            
        if model_id == DEBERTA_MODEL_ID:
            hf_id = HF_DEBERTA

        if model_id == MBERT_MODEL_ID:
            hf_id = HF_M_BERT
            
        if model_id == XLMROBERTA_MODEL_ID:
            hf_id = HF_XLM_ROBERTA

        if model_id == HINDIROBERTA_MODEL_ID:
            hf_id = HF_HIN_ROBERTA
            
        self.__load_tokenizer_model_unmasker(hf_id)
            
        print(f"{self.model_id} loaded...")

    def __load_tokenizer_model_unmasker(self, hf_id):
        self.tokenizer = AutoTokenizer.from_pretrained(hf_id,
                                                       strip_accents=False,
                                                       clean_text=False )
        self.model = AutoModelForMaskedLM.from_pretrained(hf_id)
        self.unmasker = pipeline('fill-mask', model=self.model, tokenizer=self.tokenizer)
        
        
    ## =================== MASKING - UNMASKING METHODS ======================= ##
    
    # Used to reconstruct word which is splitted on characters by a space
    def clean_token_str(self, token):
        return ''.join(token.split())

    # Masks the input sentence at random word index if mask_token_index is not given
    def mask_sentence(self, sentence, mask_token_index=None):
        print('Sentence: ', sentence)

        words = sentence.split()
        if mask_token_index == None:
            mask_token_index = np.random.randint(len(words))

        masked_word = words[mask_token_index]
        print('Masked Word: ', masked_word)

        words[mask_token_index] = self.tokenizer.mask_token
        masked_sentence = ' '.join(words)
        print('Masked Sentence: ', masked_sentence)

        return masked_word, masked_sentence

    # First mask the given sentence, then Unmasks them according to the loaded models. Result is a Pandas dataframe
    def unmask(self, sentence, mask_token_index=None):
        print(self.model_id)
        mask_token = self.tokenizer.mask_token
        masked_word, masked_sentence = self.mask_sentence(sentence, mask_token_index)

        df = pd.DataFrame(self.unmasker(masked_sentence))

        if self.model_id == BERT_MODEL_ID:
            df.token_str = df.token_str.apply(self.clean_token_str)

        return df

/home/suyogyat/research/.RNN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Defining Nepali Models

In [3]:
nepBert = TransformerModel(BERT_MODEL_ID)
nepRoberta = TransformerModel(ROBERTA_MODEL_ID)
nepDistilbert = TransformerModel(DISTILBERT_MODEL_ID)
nepDeberta = TransformerModel(DEBERTA_MODEL_ID)
mBert = TransformerModel(MBERT_MODEL_ID)
xlmRoberta = TransformerModel(XLMROBERTA_MODEL_ID)
hinRoberta = TransformerModel(HINDIROBERTA_MODEL_ID)

BERT loaded...
RoBERTa loaded...
DistilBERT loaded...
DeBERTa loaded...


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


mBERT loaded...
XLMRoBERTa loaded...
HindiRoBERTa loaded...


In [4]:
models = [nepBert, nepRoberta, nepDistilbert, nepDeberta, mBert, xlmRoberta, hinRoberta]

In [5]:
from IPython.display import display

### Example 1

In [6]:
sentence1 = 'कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ'

In [7]:
for model in models:
    df = model.unmask(sentence1, 12)
    display(df)

BERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  चिन्तित
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले [MASK] बनाएको छ


,score,token,token_str,sequence
0,0.045957,6855,निराश,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले निराश बनाएको छ
1,0.031623,2584,प्रभावित,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले प्रभावित बनाएको छ
2,0.029524,8417,चिन्तित,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
3,0.023314,3028,खुसी,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले खुसी बनाएको छ
4,0.020482,2647,कमजोर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले कमजोर बनाएको छ


RoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  चिन्तित
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ


,score,token,token_str,sequence
0,0.391272,998,सहज,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले सहज बनाएको छ
1,0.126537,596,घर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले घर बनाएको छ
2,0.092316,711,सफल,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले सफल बनाएको छ
3,0.057050,1606,असफल,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले असफल बनाएको छ
4,0.031131,730,सडक,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले सडक बनाएको छ


DistilBERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  चिन्तित
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ


...

,score,token,token_str,sequence
0,0.810321,100,[UNK],कोभिड पीडाबाट मानव समाजलाई बनाएको छ
1,0.010490,566,न,कोभिड पीडाबाट मानव समाजलाई न बनाएको छ
2,0.005811,41426,रपमा,कोभिड पीडाबाट मानव समाजलाई रपमा बनाएको छ
3,0.005600,107,"""","कोभिड पीडाबाट मानव समाजलाई "" बनाएको छ"
4,0.004687,23213,पनि,कोभिड पीडाबाट मानव समाजलाई पनि बनाएको छ


XLMRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  चिन्तित
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ


,score,token,token_str,sequence
0,0.235344,154558,बाध्य,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले बाध्य बनाएको छ
1,0.078123,112326,गम्भीर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले गम्भीर बनाएको छ
2,0.077793,87539,कमजोर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले कमजोर बनाएको छ
3,0.045162,213750,पीडा,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले पीडा बनाएको छ
4,0.045061,84877,दुःख,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले दुःख बनाएको छ


HindiRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  चिन्तित
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले <mask> बनाएको छ


,score,token,token_str,sequence
0,0.087568,348,कर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्तालेकर बनाएको छ
1,0.065402,263,क,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्तालेक बनाएको छ
2,0.058967,297,न,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले न बनाएको छ
3,0.041494,270,न,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्तालेन बनाएको छ
4,0.038912,272,स,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्तालेस बनाएको छ


**In Example 1,** 

Sentence (EN): Human society is now worried about the devastating effects of the war, without being free from the scourge of the Covid epidemic.

`BERT` captured the exact masked token 'चिन्तित' at index 2. However, first token it predicted 'निराश' is semantically correct in context of the sentence as it represents a (_state of sadness_). Out of 5, all except index 3 'खुसी' (happiness) is semantically correct.

`DistilBERT` did a great job in exactly finding the masked word with comparatively higher score value. All other tokens predicted are semantically similar with the masked token.

`RoBERTa` model's predictions are not good at all as all the predicted words did not seem to consider previous words in the sentence. Words being predicted sound correct if we just consider words after it.

`xlmRoBERTaLarge` has pretty good predictions too. However, at index 4, it predicted word 'दंग' (joyous) which does not seem to match the context. We cannot be joyous for the war.

**Winner: DistilBERT**

### Example 2 (Masked First Word)

In [8]:
random_mask_index = np.random.randint(len(sentence1.split()))
for model in models:
    df = model.unmask(sentence1, 0)
    display(df)

BERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  कोभिड
Masked Sentence:  [MASK] महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


,score,token,token_str,sequence
0,0.020728,688,तर,तर महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.013893,2592,भारतको,भारतको महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
2,0.012074,1348,हाम्रो,हाम्रो महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
3,0.010884,617,यो,यो महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
4,0.010674,5886,जातीय,जातीय महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


RoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  कोभिड
Masked Sentence:  <mask> महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


,score,token,token_str,sequence
0,0.202960,407,तर,तर महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.085310,314,एक,एक महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
2,0.078474,1047,आज,आज महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
3,0.058496,418,–,– महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
4,0.027766,401,यस,यस महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


DistilBERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  कोभिड
Masked Sentence:  <mask> महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


...

,score,token,token_str,sequence
0,0.913595,100,[UNK],पीडाबाट मानव समाजलाई बनाएको छ
1,0.011468,29357,यस,यस पीडाबाट मानव समाजलाई बनाएको छ
2,0.007694,23507,यो,यो पीडाबाट मानव समाजलाई बनाएको छ
3,0.003314,58037,यसको,यसको पीडाबाट मानव समाजलाई बनाएको छ
4,0.002627,10213,...,... पीडाबाट मानव समाजलाई बनाएको छ


XLMRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  कोभिड
Masked Sentence:  <mask> महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


,score,token,token_str,sequence
0,0.157723,13743,विश्व,विश्व महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.086191,4192,यो,यो महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
2,0.051359,13290,यस,यस महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
3,0.036176,72243,यसरी,यसरी महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
4,0.034690,67114,युद्ध,युद्ध महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


HindiRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  कोभिड
Masked Sentence:  <mask> महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


,score,token,token_str,sequence
0,0.100772,755,इस,इस महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.075876,335,इस,इस महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
2,0.029128,268,क,क महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
3,0.024786,1373,यह,यह महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
4,0.021959,1550,और,और महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ


**In Example 2 (masked first word)**

All three models have predicted same word 'तर' (but) conjunction words. 

`BERT` and `DistilBERT`'s few predictions seem intuitively accurate but `RoBERTa`'s predictions are just satisfactory.

`xlmRoBERTaLarge` seem to have better predictions with meaningful words that can be the first word of the sentence and match the context. At index 3, 'यद्यपि' is an adverb.

**Winner: xlmRoBERTaLarge**

### Example 3

In [9]:
rand_idx = np.random.randint(len(sentence1.split()))
for model in models:
    df = model.unmask(sentence1, 13)
    display(df)

BERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  बनाएको
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित [MASK] छ


,score,token,token_str,sequence
0,0.219988,2116,बनाएको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.148579,6105,बनाउँछ,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाउँछ छ
2,0.119928,5018,बनायो,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनायो छ
3,0.059056,597,छन्,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित छन् छ
4,0.044595,108,छ,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित छ छ


RoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  बनाएको
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित <mask> छ


,score,token,token_str,sequence
0,0.416456,271,ो,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तितो छ
1,0.369527,290,ै,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तितै छ
2,0.066882,265,्,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित् छ
3,0.030827,572,समय,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित समय छ
4,0.024737,264,ा,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तिता छ


DistilBERT
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  बनाएको
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित <mask> छ


...

,score,token,token_str,sequence
0,0.350055,100,[UNK],कोभिड पीडाबाट मानव समाजलाई छ
1,0.148647,67377,गरिएको,कोभिड पीडाबाट मानव समाजलाई गरिएको छ
2,0.050614,70236,ਕਿਹਾ,कोभिड पीडाबाट मानव समाजलाई ਕਿਹਾ छ
3,0.033021,591,।,कोभिड पीडाबाट मानव समाजलाई । छ
4,0.031648,59207,বলা,कोभिड पीडाबाट मानव समाजलाई বলা छ


XLMRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  बनाएको
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित <mask> छ


,score,token,token_str,sequence
0,0.836268,140968,बनाएको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
1,0.053713,112557,गरिरहेको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित गरिरहेको छ
2,0.051341,195800,पारेको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित पारेको छ
3,0.022818,70111,भइरहेको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित भइरहेको छ
4,0.007499,4080,गरेको,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित गरेको छ


HindiRoBERTa
Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित बनाएको छ
Masked Word:  बनाएको
Masked Sentence:  कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित <mask> छ


,score,token,token_str,sequence
0,0.273546,315,कर,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित कर छ
1,0.201887,386,करत,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित करत छ
2,0.065535,275,ि,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तिति छ
3,0.042549,273,ी,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तिती छ
4,0.036101,267,्,कोभिड महाव्याधिको पीडाबाट मुक्त नहुँदै मानव समाजलाई यतिबेला युद्धको विध्वंसकारी क्षतिको चिन्ताले चिन्तित् छ


**In Example 3,**

`DistilBERT`'s every top predictions are far more accurate and matches the sentence's termination words.

`BERT` although managed to predict the correct masked token with high score among its predictions, but all other predictions are simply sentence terminators (those words which appear as the last words in a sentence)

`RoBERTa`'s predictions are horrible.

`xlmRoBERTaLarge`'s predictions are competitive with `DistilBERT`'s and are very accurate too.

**Winner: DistilBERT and xlmRoBERTaLarge**

### Example 4 (Simple Sentence)

In [10]:
sentence2 = 'म राति सुत्नु अगाडि दाँत माझ्छु ।'
rand_idx = np.random.randint(len(sentence2.split()))
for model in models:
    df = model.unmask(sentence2, 3)
    display(df)

BERT
Sentence:  म राति सुत्नु अगाडि दाँत माझ्छु ।
Masked Word:  अगाडि
Masked Sentence:  म राति सुत्नु [MASK] दाँत माझ्छु ।


,score,token,token_str,sequence
0,0.132924,1126,##अघि,म राति सुत्नुअघि दाँत माझ्छु ।
1,0.045852,37276,##पर्दा,म राति सुत्नुपर्दा दाँत माझ्छु ।
2,0.035214,3509,##होस्,म राति सुत्नुहोस् दाँत माझ्छु ।
3,0.032988,17004,##भयो,म राति सुत्नुभयो दाँत माझ्छु ।
4,0.032079,2836,##पर्ने,म राति सुत्नुपर्ने दाँत माझ्छु ।


RoBERTa
Sentence:  म राति सुत्नु अगाडि दाँत माझ्छु ।
Masked Word:  अगाडि
Masked Sentence:  म राति सुत्नु <mask> दाँत माझ्छु ।


,score,token,token_str,sequence
0,0.347353,296,।,म राति सुत्नु । दाँत माझ्छु ।
1,0.237034,292,र,म राति सुत्नु र दाँत माझ्छु ।
2,0.054549,470,भए,म राति सुत्नु भए दाँत माझ्छु ।
3,0.047116,294,म,म राति सुत्नु म दाँत माझ्छु ।
4,0.037946,313,त,म राति सुत्नु त दाँत माझ्छु ।


DistilBERT
Sentence:  म राति सुत्नु अगाडि दाँत माझ्छु ।
Masked Word:  अगाडि
Masked Sentence:  म राति सुत्नु <mask> दाँत माझ्छु ।


...

,score,token,token_str,sequence
0,0.528235,100,[UNK],म राति ।
1,0.174039,591,।,म राति । ।
2,0.064721,117,",","म राति, ।"
3,0.043626,170,|,म राति | ।
4,0.023824,106,!,म राति! ।


XLMRoBERTa
Sentence:  म राति सुत्नु अगाडि दाँत माझ्छु ।
Masked Word:  अगाडि
Masked Sentence:  म राति सुत्नु <mask> दाँत माझ्छु ।


,score,token,token_str,sequence
0,0.161168,356,को,म राति सुत्नुको दाँत माझ्छु ।
1,0.128198,411,मा,म राति सुत्नुमा दाँत माझ्छु ।
2,0.102966,100260,अघि,म राति सुत्नुअघि दाँत माझ्छु ।
3,0.069714,37678,अघि,म राति सुत्नु अघि दाँत माझ्छु ।
4,0.045214,1472,स,म राति सुत्नुस दाँत माझ्छु ।


HindiRoBERTa
Sentence:  म राति सुत्नु अगाडि दाँत माझ्छु ।
Masked Word:  अगाडि
Masked Sentence:  म राति सुत्नु <mask> दाँत माझ्छु ।


,score,token,token_str,sequence
0,0.108892,279,म,म राति सुत्नुम दाँत माझ्छु ।
1,0.068072,290,म,म राति सुत्नु म दाँत माझ्छु ।
2,0.060096,265,र,म राति सुत्नुर दाँत माझ्छु ।
3,0.058133,263,क,म राति सुत्नुक दाँत माझ्छु ।
4,0.054825,324,और,म राति सुत्नु और दाँत माझ्छु ।


**In Example 4,**

Sentence (EN): I brush my teeth before I sleep.

`BERT` has semantically correct prediction.

`DistilBERT` and `xlmRoBERTaLarge` has almost all predictions accurate with the context. All predicted words has the meanings corresponding to (before).

`RoBERTa` doesn't have accurate predictions.

**Winner: DistilBERT and xlmRoBERTaLarge**

### Example 5 (Country / Destination)

In [11]:
sentence3 = 'नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।'
rand_idx = np.random.randint(len(sentence3.split()))
for model in models:
    df = model.unmask(sentence3, 5)
    display(df)

BERT
Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।
Masked Word:  श्रीलंका
Masked Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार [MASK] प्रस्थान गर्दैछ ।


,score,token,token_str,sequence
0,0.060666,4420,दिल्ली,नेपाली टोली प्रशिक्षणका लागि शनिबार दिल्ली प्रस्थान गर्दैछ ।
1,0.036525,3735,मलेसिया,नेपाली टोली प्रशिक्षणका लागि शनिबार मलेसिया प्रस्थान गर्दैछ ।
2,0.036245,897,भारत,नेपाली टोली प्रशिक्षणका लागि शनिबार भारत प्रस्थान गर्दैछ ।
3,0.029930,599,नेपाल,नेपाली टोली प्रशिक्षणका लागि शनिबार नेपाल प्रस्थान गर्दैछ ।
4,0.029095,2186,पोखरा,नेपाली टोली प्रशिक्षणका लागि शनिबार पोखरा प्रस्थान गर्दैछ ।


RoBERTa
Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।
Masked Word:  श्रीलंका
Masked Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान गर्दैछ ।


,score,token,token_str,sequence
0,0.380460,508,आज,नेपाली टोली प्रशिक्षणका लागि शनिबार आज प्रस्थान गर्दैछ ।
1,0.121045,649,थप,नेपाली टोली प्रशिक्षणका लागि शनिबार थप प्रस्थान गर्दैछ ।
2,0.067413,370,एक,नेपाली टोली प्रशिक्षणका लागि शनिबार एक प्रस्थान गर्दैछ ।
3,0.037404,270,े,नेपाली टोली प्रशिक्षणका लागि शनिबारे प्रस्थान गर्दैछ ।
4,0.027709,292,र,नेपाली टोली प्रशिक्षणका लागि शनिबार र प्रस्थान गर्दैछ ।


DistilBERT
Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।
Masked Word:  श्रीलंका
Masked Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान गर्दैछ ।


...

,score,token,token_str,sequence
0,0.505891,591,।,टोली लागि शनिबार । ।
1,0.111474,117,",","टोली लागि शनिबार, ।"
2,0.048408,100,[UNK],टोली लागि शनिबार ।
3,0.023661,119,.,टोली लागि शनिबार. ।
4,0.022215,118,-,टोली लागि शनिबार - ।


XLMRoBERTa
Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।
Masked Word:  श्रीलंका
Masked Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान गर्दैछ ।


,score,token,token_str,sequence
0,0.183484,156471,मलेसिया,नेपाली टोली प्रशिक्षणका लागि शनिबार मलेसिया प्रस्थान गर्दैछ ।
1,0.147286,4702,नेपाल,नेपाली टोली प्रशिक्षणका लागि शनिबार नेपाल प्रस्थान गर्दैछ ।
2,0.120514,23263,चीन,नेपाली टोली प्रशिक्षणका लागि शनिबार चीन प्रस्थान गर्दैछ ।
3,0.093826,135921,स्वदेश,नेपाली टोली प्रशिक्षणका लागि शनिबार स्वदेश प्रस्थान गर्दैछ ।
4,0.073144,9680,काठमाडौं,नेपाली टोली प्रशिक्षणका लागि शनिबार काठमाडौं प्रस्थान गर्दैछ ।


HindiRoBERTa
Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान गर्दैछ ।
Masked Word:  श्रीलंका
Masked Sentence:  नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान गर्दैछ ।


,score,token,token_str,sequence
0,0.269306,273,ी,नेपाली टोली प्रशिक्षणका लागि शनिबारी प्रस्थान गर्दैछ ।
1,0.090576,275,ि,नेपाली टोली प्रशिक्षणका लागि शनिबारि प्रस्थान गर्दैछ ।
2,0.052722,338,ां,नेपाली टोली प्रशिक्षणका लागि शनिबारां प्रस्थान गर्दैछ ।
3,0.051291,292,ै,नेपाली टोली प्रशिक्षणका लागि शनिबारै प्रस्थान गर्दैछ ।
4,0.044734,266,े,नेपाली टोली प्रशिक्षणका लागि शनिबारे प्रस्थान गर्दैछ ।


**In Example 5,**

Sentence (EN): The Nepali team is leaving for Sri Lanka on Saturday for training.

Here the masked token is the country name - Sri Lanka.

`BERT` predicted (Delhi, Pokhara, Malaysia, Nepal, India). It managed to return all destinations where the team could go for training.

`DistilBERT` is quite similar as it predicted (Thailand, Bangaladesh, London) as the destination names. However, the first prediction has a meaning of (towards that) which sounds correct as per the context too.

`RoBERTa`'s first prediction is correct semantically, but others are not. It also did not manage to capture any possible destinations / locations.

`xlmRoBERTaLarge` predicted (Malaysia, India, UAE, America, Sri Lanka). All the top predictions are some countries and even has the correct mask token in them.

**Winner: BERT, DistilBERT, xlmRoBERTaLarge**

### Example 6 (Added a sentence before Example 5 which has a name of capital city)

In [12]:
sentence4 = 'अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence4, 1)
    display(df)

BERT
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  महिना
Masked Sentence:  अर्को [MASK] नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.198583,1632,खेलमा,अर्को खेलमा नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
1,0.099789,662,वर्ष,अर्को वर्ष नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
2,0.096261,800,दिन,अर्को दिन नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
3,0.064732,2750,चरणमा,अर्को चरणमा नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
4,0.031531,1290,पटक,अर्को पटक नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


RoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  महिना
Masked Sentence:  अर्को <mask> नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.392081,370,एक,अर्को एक नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
1,0.140371,644,पटक,अर्को पटक नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
2,0.043830,572,समय,अर्को समय नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
3,0.042779,880,पटक,अर्कोपटक नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
4,0.030174,292,र,अर्को र नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


DistilBERT
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  महिना
Masked Sentence:  अर्को <mask> नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


...

,score,token,token_str,sequence
0,0.869350,100,[UNK],। टोली लागि शनिबार छ ।
1,0.014503,591,।,। । टोली लागि शनिबार छ ।
2,0.010579,117,",",", । टोली लागि शनिबार छ ।"
3,0.007898,118,-,- । टोली लागि शनिबार छ ।
4,0.007759,131,:,: । टोली लागि शनिबार छ ।


XLMRoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  महिना
Masked Sentence:  अर्को <mask> नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.415480,3927,दिन,अर्को दिन नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
1,0.200442,76380,खेलमा,अर्को खेलमा नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
2,0.116073,111418,साता,अर्को साता नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
3,0.064409,179321,हप्ता,अर्को हप्ता नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
4,0.021388,92657,शुक्रबार,अर्को शुक्रबार नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


HindiRoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  महिना
Masked Sentence:  अर्को <mask> नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.309082,299,ट,अर्कोट नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
1,0.079236,312,ड,अर्कोड नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
2,0.058944,281,ल,अर्कोल नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
3,0.053184,270,न,अर्कोन नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
4,0.052202,263,क,अर्कोक नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


**In Example 6,**

Sentence (EN): Nepali players are going to Colombo next month. The Nepali team is leaving for Sri Lanka on Saturday for training.

Masked token is 'महिना' (month)

`BERT`'s predictions are semantically correct (next game, next year, next day, next season, next time) . It has given words which not only correspond with the time of the year but also some other correct words like next game, or next season, or next time.

`DistilBERT`'s most predictions are based on time of the year like (next week, next month, next year). It also predicted next game. It managed to capture the exact masked token too at index 3.

`RoBERTa`'s most predictions are not semantically correct. However, prediction at index 1 and 3 seems to be sounding correct.

`xlmRoBERTaLarge` predicted (week, day, saturday, week, friday). The way it captured weekday name could be captured from the sentence following it.

**Winner: BERT, DistilBERT, xlmRoBERTaLarge**

### Example 7 (Can they predict country given its capital?)

In [13]:
sentence4 = 'अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence4, 12)
    display(df)

BERT
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  श्रीलंका
Masked Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार [MASK] प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.064686,4420,दिल्ली,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार दिल्ली प्रस्थान जाँदै छ ।
1,0.027709,2186,पोखरा,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार पोखरा प्रस्थान जाँदै छ ।
2,0.027292,897,भारत,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार भारत प्रस्थान जाँदै छ ।
3,0.024065,3735,मलेसिया,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार मलेसिया प्रस्थान जाँदै छ ।
4,0.022500,599,नेपाल,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार नेपाल प्रस्थान जाँदै छ ।


RoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  श्रीलंका
Masked Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.359789,370,एक,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार एक प्रस्थान जाँदै छ ।
1,0.205959,508,आज,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार आज प्रस्थान जाँदै छ ।
2,0.056670,649,थप,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार थप प्रस्थान जाँदै छ ।
3,0.039137,292,र,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार र प्रस्थान जाँदै छ ।
4,0.022017,16,",","अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार, प्रस्थान जाँदै छ ।"


DistilBERT
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  श्रीलंका
Masked Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान जाँदै छ ।


...

,score,token,token_str,sequence
0,0.210838,100,[UNK],महिना । टोली लागि शनिबार छ ।
1,0.116180,12572,##मा,महिना । टोली लागि शनिबारमा छ ।
2,0.053649,12500,##को,महिना । टोली लागि शनिबारको छ ।
3,0.035393,31984,दिन,महिना । टोली लागि शनिबार दिन छ ।
4,0.034458,10824,##ी,महिना । टोली लागि शनिबारी छ ।


XLMRoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  श्रीलंका
Masked Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.159444,23263,चीन,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार चीन प्रस्थान जाँदै छ ।
1,0.140750,156471,मलेसिया,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार मलेसिया प्रस्थान जाँदै छ ।
2,0.088576,4702,नेपाल,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार नेपाल प्रस्थान जाँदै छ ।
3,0.048171,3946,भारत,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार भारत प्रस्थान जाँदै छ ।
4,0.044172,173006,श्रीलंका,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।


HindiRoBERTa
Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार श्रीलंका प्रस्थान जाँदै छ ।
Masked Word:  श्रीलंका
Masked Sentence:  अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार <mask> प्रस्थान जाँदै छ ।


,score,token,token_str,sequence
0,0.203670,273,ी,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबारी प्रस्थान जाँदै छ ।
1,0.070089,275,ि,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबारि प्रस्थान जाँदै छ ।
2,0.051876,266,े,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबारे प्रस्थान जाँदै छ ।
3,0.043384,794,१,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार १ प्रस्थान जाँदै छ ।
4,0.036550,453,समय,अर्को महिना नेपाली खेलाडिहरु कोलोम्बो जाँदैछन् । नेपाली टोली प्रशिक्षणका लागि शनिबार समय प्रस्थान जाँदै छ ।


Could not do for these sentences. Let's try with a simple country-capital sentence

### Example 8 (Country-Capital sentence)

In [14]:
sentence5 = 'श्रीलंकाको राजधानी कोलोम्बो हो ।'
rand_idx = 2
for model in models:
    df = model.unmask(sentence5, rand_idx)
    display(df)

BERT
Sentence:  श्रीलंकाको राजधानी कोलोम्बो हो ।
Masked Word:  कोलोम्बो
Masked Sentence:  श्रीलंकाको राजधानी [MASK] हो ।


,score,token,token_str,sequence
0,0.135174,557,पनि,श्रीलंकाको राजधानी पनि हो ।
1,0.031390,2793,राजधानी,श्रीलंकाको राजधानी राजधानी हो ।
2,0.031236,1633,यही,श्रीलंकाको राजधानी यही हो ।
3,0.019709,703,नै,श्रीलंकाको राजधानी नै हो ।
4,0.017806,859,प्रदेश,श्रीलंकाको राजधानी प्रदेश हो ।


RoBERTa
Sentence:  श्रीलंकाको राजधानी कोलोम्बो हो ।
Masked Word:  कोलोम्बो
Masked Sentence:  श्रीलंकाको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.461069,370,एक,श्रीलंकाको राजधानी एक हो ।
1,0.043161,313,त,श्रीलंकाको राजधानी त हो ।
2,0.035551,572,समय,श्रीलंकाको राजधानी समय हो ।
3,0.024992,1036,फरक,श्रीलंकाको राजधानी फरक हो ।
4,0.017473,1272,सहर,श्रीलंकाको राजधानी सहर हो ।


DistilBERT
Sentence:  श्रीलंकाको राजधानी कोलोम्बो हो ।
Masked Word:  कोलोम्बो
Masked Sentence:  श्रीलंकाको राजधानी <mask> हो ।


...

,score,token,token_str,sequence
0,0.160388,29986,नपाल,राजधानी नपाल हो ।
1,0.083284,42562,दिलली,राजधानी दिलली हो ।
2,0.082242,60485,राजधानी,राजधानी राजधानी हो ।
3,0.054120,32021,शहर,राजधानी शहर हो ।
4,0.044138,23213,पनि,राजधानी पनि हो ।


XLMRoBERTa
Sentence:  श्रीलंकाको राजधानी कोलोम्बो हो ।
Masked Word:  कोलोम्बो
Masked Sentence:  श्रीलंकाको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.205447,30767,काठमाडौँ,श्रीलंकाको राजधानी काठमाडौँ हो ।
1,0.114152,9680,काठमाडौं,श्रीलंकाको राजधानी काठमाडौं हो ।
2,0.077531,14420,दिल्ली,श्रीलंकाको राजधानी दिल्ली हो ।
3,0.045706,48886,राजधानी,श्रीलंकाको राजधानी राजधानी हो ।
4,0.043538,4702,नेपाल,श्रीलंकाको राजधानी नेपाल हो ।


HindiRoBERTa
Sentence:  श्रीलंकाको राजधानी कोलोम्बो हो ।
Masked Word:  कोलोम्बो
Masked Sentence:  श्रीलंकाको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.326456,297,न,श्रीलंकाको राजधानी न हो ।
1,0.193127,571,अलग,श्रीलंकाको राजधानी अलग हो ।
2,0.059137,516,कल,श्रीलंकाको राजधानी कल हो ।
3,0.037944,626,शहर,श्रीलंकाको राजधानी शहर हो ।
4,0.030647,672,कब,श्रीलंकाको राजधानी कब हो ।


**In Example 8,**

Sentence (EN): Sri Lanka's capital is Colombo.

Masked token: Colombo

`BERT` did not capture the exact capital name. However, at index 0, 2 and 3 the sentences sound correct.

`DistilBERT` surprisingly predicted the correct capital name as Colombo. It's other predictions also denote name of cities (Kathmandu, City, Janakpur, Kapilvastu)

`RoBERTa`'s predictions are horrible again.

`xlmRoBERTaLarge` did not manage to capture the correct masked token. However, it's predictions are syntactically correct.

**Winner: DistilBERT**

Let's try few more examples of (Country-Capital) simple sentences

Thailand's capital is Bangkok

In [15]:
sentence6 = 'थाइल्याण्डको राजधानी बैंकक हो ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence6, 2)
    display(df)

BERT
Sentence:  थाइल्याण्डको राजधानी बैंकक हो ।
Masked Word:  बैंकक
Masked Sentence:  थाइल्याण्डको राजधानी [MASK] हो ।


,score,token,token_str,sequence
0,0.117888,557,पनि,थाइल्याण्डको राजधानी पनि हो ।
1,0.035787,2793,राजधानी,थाइल्याण्डको राजधानी राजधानी हो ।
2,0.023385,1633,यही,थाइल्याण्डको राजधानी यही हो ।
3,0.021396,2186,पोखरा,थाइल्याण्डको राजधानी पोखरा हो ।
4,0.018002,859,प्रदेश,थाइल्याण्डको राजधानी प्रदेश हो ।


RoBERTa
Sentence:  थाइल्याण्डको राजधानी बैंकक हो ।
Masked Word:  बैंकक
Masked Sentence:  थाइल्याण्डको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.466835,370,एक,थाइल्याण्डको राजधानी एक हो ।
1,0.046690,313,त,थाइल्याण्डको राजधानी त हो ।
2,0.033155,572,समय,थाइल्याण्डको राजधानी समय हो ।
3,0.021578,1272,सहर,थाइल्याण्डको राजधानी सहर हो ।
4,0.021470,1202,शहर,थाइल्याण्डको राजधानी शहर हो ।


DistilBERT
Sentence:  थाइल्याण्डको राजधानी बैंकक हो ।
Masked Word:  बैंकक
Masked Sentence:  थाइल्याण्डको राजधानी <mask> हो ।


...

,score,token,token_str,sequence
0,0.160388,29986,नपाल,राजधानी नपाल हो ।
1,0.083284,42562,दिलली,राजधानी दिलली हो ।
2,0.082242,60485,राजधानी,राजधानी राजधानी हो ।
3,0.054120,32021,शहर,राजधानी शहर हो ।
4,0.044138,23213,पनि,राजधानी पनि हो ।


XLMRoBERTa
Sentence:  थाइल्याण्डको राजधानी बैंकक हो ।
Masked Word:  बैंकक
Masked Sentence:  थाइल्याण्डको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.107306,30767,काठमाडौँ,थाइल्याण्डको राजधानी काठमाडौँ हो ।
1,0.061254,154512,काठमाण्डौ,थाइल्याण्डको राजधानी काठमाण्डौ हो ।
2,0.060455,9680,काठमाडौं,थाइल्याण्डको राजधानी काठमाडौं हो ।
3,0.054368,4702,नेपाल,थाइल्याण्डको राजधानी नेपाल हो ।
4,0.045404,33171,यही,थाइल्याण्डको राजधानी यही हो ।


HindiRoBERTa
Sentence:  थाइल्याण्डको राजधानी बैंकक हो ।
Masked Word:  बैंकक
Masked Sentence:  थाइल्याण्डको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.404408,626,शहर,थाइल्याण्डको राजधानी शहर हो ।
1,0.215116,297,न,थाइल्याण्डको राजधानी न हो ।
2,0.083227,672,कब,थाइल्याण्डको राजधानी कब हो ।
3,0.037590,765,नगर,थाइल्याण्डको राजधानी नगर हो ।
4,0.030307,324,और,थाइल्याण्डको राजधानी और हो ।


`BERT`'s few predictions give syntactically correct sentences although it did not capture correct token.

`DistilBERT` shows Kuala Lumpur as the highest scorer for capital of Thailand. However, it has Bangkok at index 2.

`RoBERTa` did not predict any specific locations at all.

`xlmRoBERTaLarge` shows 'Bangkok' with the highest score, which is the correct answer, but then token is in English. It also managed to capture some other location of Thailand i.e. 'Phuket' at index 3.

**Winner: xlmRoBERTaLarge**

India's capital is New Delhi

In [16]:
sentence7 = 'भारतको राजधानी नयाँदिल्ली हो ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence7, 2)
    display(df)

BERT
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी [MASK] हो ।


,score,token,token_str,sequence
0,0.142466,557,पनि,भारतको राजधानी पनि हो ।
1,0.041588,2793,राजधानी,भारतको राजधानी राजधानी हो ।
2,0.026896,703,नै,भारतको राजधानी नै हो ।
3,0.021884,1633,यही,भारतको राजधानी यही हो ।
4,0.018922,859,प्रदेश,भारतको राजधानी प्रदेश हो ।


RoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.327671,370,एक,भारतको राजधानी एक हो ।
1,0.069961,313,त,भारतको राजधानी त हो ।
2,0.050587,1272,सहर,भारतको राजधानी सहर हो ।
3,0.039997,1202,शहर,भारतको राजधानी शहर हो ।
4,0.029238,1036,फरक,भारतको राजधानी फरक हो ।


DistilBERT
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


...

,score,token,token_str,sequence
0,0.856558,42562,दिलली,भारतको राजधानी दिलली हो ।
1,0.027935,63526,मबई,भारतको राजधानी मबई हो ।
2,0.018811,41733,पण,भारतको राजधानी पण हो ।
3,0.017361,73865,अलमोडा,भारतको राजधानी अलमोडा हो ।
4,0.007053,96613,राजसथान,भारतको राजधानी राजसथान हो ।


XLMRoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.692451,14420,दिल्ली,भारतको राजधानी दिल्ली हो ।
1,0.035369,193999,कोलकाता,भारतको राजधानी कोलकाता हो ।
2,0.023657,69499,અમદાવાદ,भारतको राजधानी અમદાવાદ हो ।
3,0.017780,218440,ગાંધીનગર,भारतको राजधानी ગાંધીનગર हो ।
4,0.014798,176279,पटना,भारतको राजधानी पटना हो ।


HindiRoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.294365,297,न,भारतको राजधानी न हो ।
1,0.150879,626,शहर,भारतको राजधानी शहर हो ।
2,0.056496,324,और,भारतको राजधानी और हो ।
3,0.018658,765,नगर,भारतको राजधानी नगर हो ।
4,0.016832,325,पर,भारतको राजधानी पर हो ।


`DistilBERT` accurately shows (Delhi, New Delhi) as capital of India at index 0 and 1 respectively. As earlier, other predictions of this model are too syntactically correct.

`xlmRoBERTaLarge` shows Delhi in Devanagari with the highest score and Delhi in English to be the next token. Surprisingly, other tokens it managed to predict are all the cities of India.

In [17]:
sentence8 = 'भारतको राजधानी नयाँदिल्ली हो ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence7, 2)
    display(df)

BERT
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी [MASK] हो ।


,score,token,token_str,sequence
0,0.142466,557,पनि,भारतको राजधानी पनि हो ।
1,0.041588,2793,राजधानी,भारतको राजधानी राजधानी हो ।
2,0.026896,703,नै,भारतको राजधानी नै हो ।
3,0.021884,1633,यही,भारतको राजधानी यही हो ।
4,0.018922,859,प्रदेश,भारतको राजधानी प्रदेश हो ।


RoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.327671,370,एक,भारतको राजधानी एक हो ।
1,0.069961,313,त,भारतको राजधानी त हो ।
2,0.050587,1272,सहर,भारतको राजधानी सहर हो ।
3,0.039997,1202,शहर,भारतको राजधानी शहर हो ।
4,0.029238,1036,फरक,भारतको राजधानी फरक हो ।


DistilBERT
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


...

,score,token,token_str,sequence
0,0.856558,42562,दिलली,भारतको राजधानी दिलली हो ।
1,0.027935,63526,मबई,भारतको राजधानी मबई हो ।
2,0.018811,41733,पण,भारतको राजधानी पण हो ।
3,0.017361,73865,अलमोडा,भारतको राजधानी अलमोडा हो ।
4,0.007053,96613,राजसथान,भारतको राजधानी राजसथान हो ।


XLMRoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.692451,14420,दिल्ली,भारतको राजधानी दिल्ली हो ।
1,0.035369,193999,कोलकाता,भारतको राजधानी कोलकाता हो ।
2,0.023657,69499,અમદાવાદ,भारतको राजधानी અમદાવાદ हो ।
3,0.017780,218440,ગાંધીનગર,भारतको राजधानी ગાંધીનગર हो ।
4,0.014798,176279,पटना,भारतको राजधानी पटना हो ।


HindiRoBERTa
Sentence:  भारतको राजधानी नयाँदिल्ली हो ।
Masked Word:  नयाँदिल्ली
Masked Sentence:  भारतको राजधानी <mask> हो ।


,score,token,token_str,sequence
0,0.294365,297,न,भारतको राजधानी न हो ।
1,0.150879,626,शहर,भारतको राजधानी शहर हो ।
2,0.056496,324,और,भारतको राजधानी और हो ।
3,0.018658,765,नगर,भारतको राजधानी नगर हो ।
4,0.016832,325,पर,भारतको राजधानी पर हो ।


## Model Comparison on Masked Language Modeling Task

**TODO:**

- Document how each of the model is trained, on which dataset, number of sentences
- Document the model architectures used while pretraining (number of layers and dimensions) each of the models



Out of the three models chosen, based on our personal intuition on Nepali Language, 

`DistilBERT` seems to outperform pretrained models like 'BERT' and 'RoBERTa' by a very good margin. 

`BERT` comes next as it could capture accurate semantically similar words based on the context of the sentence.

`RoBERTa` model pretrained for Nepali seems to have some serious issues in capturing both semantic and syntactic meanings of words. It may be due to the amount of data used for training or lack of training epochs. Or it may perform better in other use cases?

`xlmRoBERTaLarge` is competitive with `DistilBERT` when compared with Nepali sentences in MLM task. Moreover, it does a very good job in most of the sentence use cases.

In [18]:
sentence9 = 'मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।'
rand_idx = np.random.randint(len(sentence4.split()))
for model in models:
    df = model.unmask(sentence9, -2)
    display(df)

BERT
Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।
Masked Word:  फाइदा
Masked Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई [MASK] गर्छ।


,score,token,token_str,sequence
0,0.026675,557,पनि,मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन् । उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन् । उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन् जसले नियम तोड्न सक्छ आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई पनि गर्छ ।
1,0.026602,771,मात्र,मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन् । उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन् । उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन् जसले नियम तोड्न सक्छ आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई मात्र गर्छ ।
2,0.019317,128,र,मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन् । उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन् । उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन् जसले नियम तोड्न सक्छ आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई र गर्छ ।
3,0.016614,621,के,मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन् । उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन् । उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन् जसले नियम तोड्न सक्छ आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई के गर्छ ।
4,0.015534,1458,किन,मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन् । उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन् । उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन् जसले नियम तोड्न सक्छ आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई किन गर्छ ।


RoBERTa
Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।
Masked Word:  फाइदा
Masked Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई <mask> गर्छ

,score,token,token_str,sequence
0,0.266240,1302,असर,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई असर गर्छ।"
1,0.162192,421,कम,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई कम गर्छ।"
2,0.073644,4156,मनन,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई मनन गर्छ।"
3,0.064375,912,खबर,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई खबर गर्छ।"
4,0.049924,1381,पहल,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई पहल गर्छ।"


DistilBERT
Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।
Masked Word:  फाइदा
Masked Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई <mask> ग

...

,score,token,token_str,sequence
0,0.098229,27284,परापत,"गतिमा विघटन नियमनकारी बीचमा अवसरको फाइदा उठाएका । एउटा सानो फौज र, वकिल तथा एउटा ।, नियम, लागि काम शोषण र, अवरोध उपहास र मानवतालाई परापत ।"
1,0.073410,100,[UNK],"गतिमा विघटन नियमनकारी बीचमा अवसरको फाइदा उठाएका । एउटा सानो फौज र, वकिल तथा एउटा ।, नियम, लागि काम शोषण र, अवरोध उपहास र मानवतालाई ।"
2,0.036848,35931,सथापित,"गतिमा विघटन नियमनकारी बीचमा अवसरको फाइदा उठाएका । एउटा सानो फौज र, वकिल तथा एउटा ।, नियम, लागि काम शोषण र, अवरोध उपहास र मानवतालाई सथापित ।"
3,0.030605,26106,गरन,"गतिमा विघटन नियमनकारी बीचमा अवसरको फाइदा उठाएका । एउटा सानो फौज र, वकिल तथा एउटा ।, नियम, लागि काम शोषण र, अवरोध उपहास र मानवतालाई गरन ।"
4,0.023436,55898,आफनो,"गतिमा विघटन नियमनकारी बीचमा अवसरको फाइदा उठाएका । एउटा सानो फौज र, वकिल तथा एउटा ।, नियम, लागि काम शोषण र, अवरोध उपहास र मानवतालाई आफनो ।"


XLMRoBERTa
Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।
Masked Word:  फाइदा
Masked Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई <mask> ग

,score,token,token_str,sequence
0,0.187668,86773,प्रभावित,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई प्रभावित गर्छ।"
1,0.140219,149285,नष्ट,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई नष्ट गर्छ।"
2,0.119885,201846,अपमान,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई अपमान गर्छ।"
3,0.031810,157781,फाइदा,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।"
4,0.030747,87539,कमजोर,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई कमजोर गर्छ।"


HindiRoBERTa
Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई फाइदा गर्छ।
Masked Word:  फाइदा
Masked Sentence:  मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई <mask>

,score,token,token_str,sequence
0,0.065805,284,य,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाईय गर्छ।"
1,0.057816,270,न,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाईन गर्छ।"
2,0.030508,267,्,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाई् गर्छ।"
3,0.022861,288,द,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाईद गर्छ।"
4,0.022654,280,त,"मस्कले तीव्र गतिमा विघटन भइरहेको नियमनकारी राज्य संयन्त्रका बीचमा उदाउँदो अवसरको फाइदा उठाएका छन्। उनले लगानीकर्ताहरुको एउटा सानो फौज र लबिइस्ट, वकिल तथा प्रशंसकहरुको एउटा जत्था जुटाएका छन्। उनले आफूलाई प्रविधिमा प्रतिभावान त्यस्ता व्यक्तिका रुपमा उभ्याउन खोजिरहेका छन्, जसले नियम तोड्न सक्छ, आफ्ना लागि काम गर्नेहरुलाई शोषण गर्न र निचोर्न सक्छ, अवरोध गर्न खोज्नेहरुलाई उपहास गर्न सक्छ र आफ्नो सम्पत्तिलाई मनलाग्दी ढंगले खर्च गर्न सक्छ किनभने त्यसले मानवतालाईत गर्छ।"


In [20]:
sentence10 = 'एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।'
for model in models:
    df = model.unmask(sentence10, 4)
    display(df)

BERT
Sentence:  एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
Masked Word:  आमा
Masked Sentence:  एकदिन राजाले जेठी रानी [MASK] बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


,score,token,token_str,sequence
0,0.044026,2657,राजा,एकदिन राजाले जेठी रानी राजा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी जेठी रानीको सन्तानले राजपाट पाउला भनेर चिन्तित भइन् ।
1,0.032298,2089,आमा,एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी जेठी रानीको सन्तानले राजपाट पाउला भनेर चिन्तित भइन् ।
2,0.032277,703,नै,एकदिन राजाले जेठी रानी नै बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी जेठी रानीको सन्तानले राजपाट पाउला भनेर चिन्तित भइन् ।
3,0.029561,2707,छोरी,एकदिन राजाले जेठी रानी छोरी बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी जेठी रानीको सन्तानले राजपाट पाउला भनेर चिन्तित भइन् ।
4,0.027827,850,घर,एकदिन राजाले जेठी रानी घर बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी जेठी रानीको सन्तानले राजपाट पाउला भनेर चिन्तित भइन् ।


RoBERTa
Sentence:  एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
Masked Word:  आमा
Masked Sentence:  एकदिन राजाले जेठी रानी <mask> बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


,score,token,token_str,sequence
0,0.339187,596,घर,एकदिन राजाले जेठी रानी घर बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
1,0.178087,292,र,एकदिन राजाले जेठी रानी र बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
2,0.026930,1082,बम,एकदिन राजाले जेठी रानी बम बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
3,0.026497,266,र,एकदिन राजाले जेठी रानीर बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
4,0.024713,1344,मगर,एकदिन राजाले जेठी रानी मगर बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


DistilBERT
Sentence:  एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
Masked Word:  आमा
Masked Sentence:  एकदिन राजाले जेठी रानी <mask> बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


...

,score,token,token_str,sequence
0,0.401107,12500,##को,एकदिन रानीको खबर पाए । राजा भए । तर रानी रानीको राजपाट पाउला ।
1,0.165578,28669,##लाई,एकदिन रानीलाई खबर पाए । राजा भए । तर रानी रानीको राजपाट पाउला ।
2,0.095389,11468,##ल,एकदिन रानील खबर पाए । राजा भए । तर रानी रानीको राजपाट पाउला ।
3,0.041190,566,न,एकदिन रानी न खबर पाए । राजा भए । तर रानी रानीको राजपाट पाउला ।
4,0.030692,15132,##का,एकदिन रानीका खबर पाए । राजा भए । तर रानी रानीको राजपाट पाउला ।


XLMRoBERTa
Sentence:  एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
Masked Word:  आमा
Masked Sentence:  एकदिन राजाले जेठी रानी <mask> बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


,score,token,token_str,sequence
0,0.287651,79032,आमा,एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
1,0.190123,116886,छोरी,एकदिन राजाले जेठी रानी छोरी बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
2,0.140431,202226,गर्भवती,एकदिन राजाले जेठी रानी गर्भवती बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
3,0.036225,226497,सन्तान,एकदिन राजाले जेठी रानी सन्तान बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
4,0.030945,48134,राजा,एकदिन राजाले जेठी रानी राजा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


HindiRoBERTa
Sentence:  एकदिन राजाले जेठी रानी आमा बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
Masked Word:  आमा
Masked Sentence:  एकदिन राजाले जेठी रानी <mask> बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


,score,token,token_str,sequence
0,0.403300,263,क,एकदिन राजाले जेठी रानीक बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
1,0.021574,297,न,एकदिन राजाले जेठी रानी न बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
2,0.019672,356,इ,एकदिन राजाले जेठी रानीइ बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
3,0.017484,268,क,एकदिन राजाले जेठी रानी क बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।
4,0.013481,310,आ,एकदिन राजाले जेठी रानी आ बन्न लागेको खबर पाए । राजा निकै खुसी भए । तर कान्छी रानी ‘जेठी रानीको सन्तानले राजपाट पाउला’ भनेर चिन्तित भइन् ।


In [21]:
sentence11 = 'एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।'
for model in models:
    df = model.unmask(sentence11, 6)
    display(df)

BERT
Sentence:  एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
Masked Word:  धूर्त
Masked Sentence:  एउटा स्याल थियो । ऊ चौपट्टै [MASK] थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


,score,token,token_str,sequence
0,0.032082,611,भएको,एउटा स्याल थियो । ऊ चौपट्टै भएको थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
1,0.027109,557,पनि,एउटा स्याल थियो । ऊ चौपट्टै पनि थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
2,0.022986,841,भन्ने,एउटा स्याल थियो । ऊ चौपट्टै भन्ने थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
3,0.019778,703,नै,एउटा स्याल थियो । ऊ चौपट्टै नै थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
4,0.016529,771,मात्र,एउटा स्याल थियो । ऊ चौपट्टै मात्र थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


RoBERTa
Sentence:  एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
Masked Word:  धूर्त
Masked Sentence:  एउटा स्याल थियो । ऊ चौपट्टै <mask> थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


,score,token,token_str,sequence
0,0.124221,1599,डर,एउटा स्याल थियो । ऊ चौपट्टै डर थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
1,0.073719,1036,फरक,एउटा स्याल थियो । ऊ चौपट्टै फरक थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
2,0.061984,572,समय,एउटा स्याल थियो । ऊ चौपट्टै समय थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
3,0.036379,596,घर,एउटा स्याल थियो । ऊ चौपट्टै घर थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
4,0.028555,410,मन,एउटा स्याल थियो । ऊ चौपट्टै मन थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


DistilBERT
Sentence:  एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
Masked Word:  धूर्त
Masked Sentence:  एउटा स्याल थियो । ऊ चौपट्टै <mask> थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


...

,score,token,token_str,sequence
0,0.928653,100,[UNK],एउटा थियो । ऊ थियो । ऊ दाउमा । कतिलाई त ऊ ।
1,0.004293,71186,एउटा,एउटा थियो । ऊ एउटा थियो । ऊ दाउमा । कतिलाई त ऊ ।
2,0.003975,10213,...,एउटा थियो । ऊ... थियो । ऊ दाउमा । कतिलाई त ऊ ।
3,0.002687,23507,यो,एउटा थियो । ऊ यो थियो । ऊ दाउमा । कतिलाई त ऊ ।
4,0.002367,11384,एक,एउटा थियो । ऊ एक थियो । ऊ दाउमा । कतिलाई त ऊ ।


XLMRoBERTa
Sentence:  एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
Masked Word:  धूर्त
Masked Sentence:  एउटा स्याल थियो । ऊ चौपट्टै <mask> थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


,score,token,token_str,sequence
0,0.064052,44655,ठूलो,एउटा स्याल थियो । ऊ चौपट्टै ठूलो थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
1,0.057628,2664,भएको,एउटा स्याल थियो । ऊ चौपट्टै भएको थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
2,0.045262,356,को,एउटा स्याल थियो । ऊ चौपट्टैको थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
3,0.044297,6425,नै,एउटा स्याल थियो । ऊ चौपट्टै नै थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
4,0.040136,71177,लामो,एउटा स्याल थियो । ऊ चौपट्टै लामो थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


HindiRoBERTa
Sentence:  एउटा स्याल थियो । ऊ चौपट्टै धूर्त थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
Masked Word:  धूर्त
Masked Sentence:  एउटा स्याल थियो । ऊ चौपट्टै <mask> थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


,score,token,token_str,sequence
0,0.107658,263,क,एउटा स्याल थियो । ऊ चौपट्टैक थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
1,0.092168,281,ल,एउटा स्याल थियो । ऊ चौपट्टैल थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
2,0.070164,270,न,एउटा स्याल थियो । ऊ चौपट्टैन थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
3,0.054534,454,ओ,एउटा स्याल थियो । ऊ चौपट्टै ओ थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।
4,0.035235,265,र,एउटा स्याल थियो । ऊ चौपट्टैर थियो । ऊ सधैँ अरूलाई झुक्याएर ठग्ने दाउमा हुन्थ्यो । कतिलाई त ऊ धम्क्याएरै खानेकुरा चोर्थ्यो ।


In [22]:
sentence12 = 'सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।'
for model in models:
    df = model.unmask(sentence12, -10)
    display(df)

BERT
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला [MASK] होने लगा। और एक दिन उसकी मृत्यु हो गई।


,score,token,token_str,sequence
0,0.011243,621,के,सालों पहले एक शहर में एक चक्की वाला रहता था । उसके तीन बेटे थे । उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था । समय बीतता गया और उसके बेटे बड़े हो गए । उम्र के साथ वह चक्की वाला के होने लगा । और एक दिन उसकी मृत्यु हो गई ।
1,0.010890,589,हो,सालों पहले एक शहर में एक चक्की वाला रहता था । उसके तीन बेटे थे । उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था । समय बीतता गया और उसके बेटे बड़े हो गए । उम्र के साथ वह चक्की वाला हो होने लगा । और एक दिन उसकी मृत्यु हो गई ।
2,0.008689,1459,ले,सालों पहले एक शहर में एक चक्की वाला रहता था । उसके तीन बेटे थे । उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था । समय बीतता गया और उसके बेटे बड़े हो गए । उम्र के साथ वह चक्की वाला ले होने लगा । और एक दिन उसकी मृत्यु हो गई ।
3,0.008486,587,मा,सालों पहले एक शहर में एक चक्की वाला रहता था । उसके तीन बेटे थे । उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था । समय बीतता गया और उसके बेटे बड़े हो गए । उम्र के साथ वह चक्की वाला मा होने लगा । और एक दिन उसकी मृत्यु हो गई ।
4,0.006997,669,को,सालों पहले एक शहर में एक चक्की वाला रहता था । उसके तीन बेटे थे । उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था । समय बीतता गया और उसके बेटे बड़े हो गए । उम्र के साथ वह चक्की वाला को होने लगा । और एक दिन उसकी मृत्यु हो गई ।


RoBERTa
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला <mask> होने लगा। और एक दिन उसकी मृत्यु हो गई।


,score,token,token_str,sequence
0,0.350299,319,ई,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाई होने लगा। और एक दिन उसकी मृत्यु हो गई।
1,0.075071,307,श,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाश होने लगा। और एक दिन उसकी मृत्यु हो गई।
2,0.044358,332,इ,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाइ होने लगा। और एक दिन उसकी मृत्यु हो गई।
3,0.036399,278,स,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालास होने लगा। और एक दिन उसकी मृत्यु हो गई।
4,0.033247,292,र,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला र होने लगा। और एक दिन उसकी मृत्यु हो गई।


DistilBERT
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला <mask> होने लगा। और एक दिन उसकी मृत्यु हो गई।


...

,score,token,token_str,sequence
0,0.141760,11384,एक,एक शहर एक वाला रहता था । तीन । उन छोटा उसका लाडला था । समय बीतता गया और हो गए । साथ वह वाला एक लगा । और एक दिन उसकी हो गई ।
1,0.110127,13610,भी,एक शहर एक वाला रहता था । तीन । उन छोटा उसका लाडला था । समय बीतता गया और हो गए । साथ वह वाला भी लगा । और एक दिन उसकी हो गई ।
2,0.057835,21967,वह,एक शहर एक वाला रहता था । तीन । उन छोटा उसका लाडला था । समय बीतता गया और हो गए । साथ वह वाला वह लगा । और एक दिन उसकी हो गई ।
3,0.052534,591,।,एक शहर एक वाला रहता था । तीन । उन छोटा उसका लाडला था । समय बीतता गया और हो गए । साथ वह वाला । लगा । और एक दिन उसकी हो गई ।
4,0.047860,100,[UNK],एक शहर एक वाला रहता था । तीन । उन छोटा उसका लाडला था । समय बीतता गया और हो गए । साथ वह वाला लगा । और एक दिन उसकी हो गई ।


XLMRoBERTa
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला <mask> होने लगा। और एक दिन उसकी मृत्यु हो गई।


,score,token,token_str,sequence
0,0.460363,1780,भी,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला भी होने लगा। और एक दिन उसकी मृत्यु हो गई।
1,0.226171,30061,बड़ा,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बड़ा होने लगा। और एक दिन उसकी मृत्यु हो गई।
2,0.104011,152920,छोटा,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला छोटा होने लगा। और एक दिन उसकी मृत्यु हो गई।
3,0.017127,32473,बड़े,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बड़े होने लगा। और एक दिन उसकी मृत्यु हो गई।
4,0.016666,201422,खड़ा,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला खड़ा होने लगा। और एक दिन उसकी मृत्यु हो गई।


HindiRoBERTa
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला <mask> होने लगा। और एक दिन उसकी मृत्यु हो गई।


,score,token,token_str,sequence
0,0.178134,275,ि,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाि होने लगा। और एक दिन उसकी मृत्यु हो गई।
1,0.075419,284,य,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाय होने लगा। और एक दिन उसकी मृत्यु हो गई।
2,0.043435,267,्,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला् होने लगा। और एक दिन उसकी मृत्यु हो गई।
3,0.036234,282,ो,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाो होने लगा। और एक दिन उसकी मृत्यु हो गई।
4,0.035373,266,े,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वालाे होने लगा। और एक दिन उसकी मृत्यु हो गई।


In [23]:
l3puneHindiRoberta = TransformerModel('l3cube-pune/hindi-roberta')

l3cube-pune/hindi-roberta loaded...


In [24]:
l3puneHindiRoberta.model.config

XLMRobertaConfig {
  "_name_or_path": "l3cube-pune/hindi-roberta",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [25]:
df = l3puneHindiRoberta.unmask(sentence12, -10)
display(df)

l3cube-pune/hindi-roberta
Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बीमार होने लगा। और एक दिन उसकी मृत्यु हो गई।
Masked Word:  बीमार
Masked Sentence:  सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला <mask> होने लगा। और एक दिन उसकी मृत्यु हो गई।


,score,token,token_str,sequence
0,0.729414,30061,बड़ा,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला बड़ा होने लगा। और एक दिन उसकी मृत्यु हो गई।
1,0.149042,152920,छोटा,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला छोटा होने लगा। और एक दिन उसकी मृत्यु हो गई।
2,0.018935,1780,भी,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला भी होने लगा। और एक दिन उसकी मृत्यु हो गई।
3,0.018200,87539,कमजोर,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला कमजोर होने लगा। और एक दिन उसकी मृत्यु हो गई।
4,0.011817,8663,कम,सालों पहले एक शहर में एक चक्की वाला रहता था। उसके तीन बेटे थे। उन तीनों बेटों में से सबसे छोटा बेटा मार्की उसका लाडला था। समय बीतता गया और उसके बेटे बड़े हो गए। उम्र के साथ वह चक्की वाला कम होने लगा। और एक दिन उसकी मृत्यु हो गई।


In [26]:
from Nepali_nlp import Embeddings

ModuleNotFoundError: No module named 'Nepali_nlp'